In [1]:
%%capture
%matplotlib widget

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
# I recommend that you do not change things in this cell!
url = "https://nomad-hzb-ce.de/nomad-oasis/api/v1"
token = os.environ['NOMAD_CLIENT_ACCESS_TOKEN']
sys.path.insert(1, '../../python-scripts-c6fxKDJrSsWp1xCxON1Y7g')
from api_calls import *
from tafel_slope_functions import *
sys.path.append(os.path.abspath('..'))
from helpers import *

# Tafel Slope

This notebook is designed for uploads with multiple `CE_NOME_Chronoamperometry` files.  
It allows you to get the Tafel Slope using all CA files and optionally you can download the calculated data. 

**For more custom tafel slope scripts you can copy and edit the file `tafel_slope_jupyter.ipynb` in your own upload.**


In [3]:
# all widgets used in notebook
uploads = get_all_uploads(url, token, number_of_uploads=250)
upload_checkboxes = [widgets.Checkbox(value=False, description=upload.get('upload_name','--no-name--')) for upload in uploads]
scrollable_box = widgets.VBox(upload_checkboxes)
scrollable_area = widgets.Box([scrollable_box],
                              layout=widgets.Layout(
                                  overflow='scroll',
                                  border='1px solid gray',
                                  height='300px',
                                  width='600px',
                                  flex_flow='column',
                                  display='flex'
                              ))
select_uploads_button = widgets.Button(description='Confirm Selected Uploads', layout=widgets.Layout(width='600px',))
upload_selection_output = widgets.Output()

avg_points_box = widgets.BoundedIntText(
    value=40,
    min=1,
    max=200,
    step=1,
    description='Number of records to be averaged:',
    layout=widgets.Layout(width='300px'),
    style={'description_width': 'initial'}
)
run_analysis_button = widgets.Button(description='Run Tafel Slope Analysis', layout=widgets.Layout(width='300px',))
show_avg_calc_button = widgets.Button(description='Show Calculation of Average Current', layout=widgets.Layout(width='300px',))
clear_output_button = widgets.Button(description='Clear Output', layout=widgets.Layout(width='300px',))
analysis_output = widgets.Output()
avg_calc_output = widgets.Output()

download_button = widgets.Button(description='Prepare Data for Download', layout=widgets.Layout(width='300px'))
download_button_output = widgets.Output()

In [4]:
# widget logic
def save_selected_uploads(b):
    global upload_names, data
    upload_names = [cb.description for cb in upload_checkboxes if cb.value]
    with upload_selection_output:
        clear_output(wait=True)
        print('Collecting CA data from selected uploads. This can take a moment...')
        data = get_ca_env_upload(url, token, upload_names)
        clear_output(wait=True)
        print('Selected:', upload_names)
        print('\nNumber of found CA files:', len(data))

def run_analysis(b):
    global overview_df
    plt.close('all')
    with analysis_output:
        clear_output()
        avg_points = avg_points_box.value
        overview_df = get_overview_df(data, avg_points)
        display(overview_df)
        show_tafel_slope_plot(overview_df)

def show_avg_calc(b):
    plt.close('all')
    with avg_calc_output:
        clear_output()
        avg_points = avg_points_box.value
        plot_avg_current_calculation(overview_df, avg_points)

def on_clear_output_clicked(b):
    plt.close('all')
    with analysis_output:
        clear_output()
    with avg_calc_output:
        clear_output()
        
select_uploads_button.on_click(save_selected_uploads)
run_analysis_button.on_click(run_analysis)
show_avg_calc_button.on_click(show_avg_calc)
clear_output_button.on_click(on_clear_output_clicked)

### Select uploads

In [5]:
upload_names = []
data = []
overview_df = pd.DataFrame()

display(scrollable_area, select_uploads_button, upload_selection_output)

Box(children=(VBox(children=(Checkbox(value=False, description='KMC3 tests'), Checkbox(value=False, descriptio…

Button(description='Confirm Selected Uploads', layout=Layout(width='600px'), style=ButtonStyle())

Output()

### Run Analysis

For each file, we are looking for the **average of the last `x` recorded current values**. 
Please enter the number of recorded values to be averaged.

In [6]:
display(avg_points_box, run_analysis_button, analysis_output, show_avg_calc_button, avg_calc_output, clear_output_button)

BoundedIntText(value=40, description='Number of records to be averaged:', layout=Layout(width='300px'), max=20…

Button(description='Run Tafel Slope Analysis', layout=Layout(width='300px'), style=ButtonStyle())

Output()

Button(description='Show Calculation of Average Current', layout=Layout(width='300px'), style=ButtonStyle())

Output()

Button(description='Clear Output', layout=Layout(width='300px'), style=ButtonStyle())

### Download Data

In [7]:
def on_download_button_clicked(b):
    if overview_df.empty:
        with download_button_output:
            clear_output(wait=True)
            print('Please select another upload that contains CA data and run the analysis before downloading.')
            return
    df_list = [upload_df for _, upload_df in overview_df.groupby('upload_name')]
    file_name_list = [f'{upload_name}_tafel_slope.csv' for upload_name, _ in overview_df.groupby('upload_name')]
    with download_button_output:
        clear_output(wait=True)
        try:
            create_zip_download_link(df_list, file_name_list, 'tafel_slope_data')
            plot_list = [plt.figure(fig_num) for fig_num in plt.get_fignums()]
            fig_names = [f'plot_{fig_num}' for fig_num in plt.get_fignums()]
            create_zip_plots_link(plot_list, fig_names, 'tafel_slope_plots')
        except NameError:
            print('Please select another upload that contains CA data and run the analysis before downloading.')

download_button.on_click(on_download_button_clicked)
display(download_button, download_button_output)

Button(description='Prepare Data for Download', layout=Layout(width='300px'), style=ButtonStyle())

Output()